                                 ############# INSTRUCTIONS ##################
**See the instructions in the cell immediately below the cell that begins with "def unique_sorted_values_plus_ALL(array):" **

 Use this notebook to recover a selected scenario, version, or year (or some combination of those) from an Essbase extract file
 Multiple members per dimension can be recovered
 NOTE: Mulitiple extract files are allowed, as long as they are ALL FROM THE SAME CUBE

 You can also do comparisons based on those three dimensions 
 However, you can select only one dimension at a time for comparisons (and only two members from the selected dimension)
 NOTE:  ALL COMPARISONS OTHER THAN "BEFORE VS. AFTER" MUST OCCUR WITHIN A SINGLE EXTRACT FILE

 v3.0
 Restored all FY25 Working Forecast data from "Export_FINSTMT_NY1-NY5_All_Lev0_102120_BOD.txt" (with accuracy to 10+ decimals)
 Renamed the output file to "FINSTMT_FY25_Forecast_Working_NonZeroRows.txt" and manually loaded it into FIN_STMT 
 on 10/23/20 using the _MXL_Batch_Load load rule

 v3.1 
 Successfully tested recoveries and comparisons from AllStaff, CAPITAL, CAPT_RPT, FIN_STMT, and SWA_RPT

 v4.0  
 Added widgets for filtering
 Added ability to filter on capacity months loaded
 
 #########################################################################################
 Use the load rule MXL_Batch_Load to load files that don't involve comparisons 
 #########################################################################################
 

In [97]:
import pandas as pd
import numpy as np
import os
import datetime
import glob
import re
import openpyxl
import ipywidgets as widgets
import qgrid
import sys
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
from pandas.api.types import CategoricalDtype

In [98]:
pd.__version__    # WARNING: Version 1.3.5 was generating errors during the pandas import; 1.3.2 is an intentional downgrade

'1.3.2'

In [99]:
#pd.show_versions()

In [100]:
InteractiveShell.ast_node_interactivity = "all"  # This outputs all commands in a cell, not just the last one

In [101]:
pd.set_option('display.max_columns', 24, 'display.max_rows', 100)

In [102]:
#os.chdir('C:\\Users\\e79230\\Documents\\MyJupyterFiles') # Define the current working directory

In [103]:
os.getcwd()

'C:\\Users\\e79230\\OneDrive - Southwest Airlines\\Documents\\MyJupyterFiles\\files\\Recover_and_Compare'

In [104]:
# Get the current date and time to append to the output file names below
now = datetime.datetime.now()
current_date = str(now.year) + '-' + str(now.month).zfill(2) + '-' + str(now.day).zfill(2)

In [105]:
#  Open all extract files in the Essbase directory and concatenate them into a single file
#  Directory location:  C:\Users\e79230\Documents\MyJupyterFiles\data\essbase

#  NOTE: ALL EXTRACT FILES MUST BE FROM THE SAME CUBE

#path = r'./data/essbase' 
#path = 'C:\\Users\\e79230\\OneDrive - Southwest Airlines\\Documents\\MyJupyterFiles\\data\\essbase\\'
path = 'C:\\Users\\e79230\\OneDrive - Southwest Airlines\\Documents\\MyJupyterFiles\\'
df_list = []

all_files = glob.glob(path + "\\data\\essbase\\" + "/*.txt")  # This creates a list of the file names
#all_files[0] = './data/essbase\\Export_FINSTMT_CY_All_Lev0.txt' 

## This block works, but it doesn't allow us to capture the filename on the fly
#ess_dfs = (pd.read_csv(f, delimiter='|', header=None) for f in all_files)  # This creates a generator that contains dataframes
#for count, df in enumerate(ess_dfs):
#    df['DATE'] = str(count) # Add a Date column so we can filter on it after the dataframes have been concatenated below 
#    df_list.append(df)  #pd.concat does not work with generators, so we have to add each dataframe to a list

for f in all_files:
    file_name = re.search(r'\\(.+)',f).group(1)  # f = './data/essbase\\Export_FINSTMT_CY_All_Lev0_050420_BOD.txt'
    df = pd.read_csv(f, delimiter='|', header=None)
    df = df.iloc[:, :-1] # Drop the last column because the export file includes a pipe character at the end of every line  
    df['FILENAME'] = file_name # Add a filename column for filtering after the dataframes have been concatenated below
    df_list.append(df)  #pd.concat does not work with generators, so we have to add each dataframe to a list

    
# Determine cube name from the extract filename
if 'FIN' in all_files[0]:
    cube_name = 'FINSTMT'
elif 'SWA' in all_files[0]:
    cube_name = 'SWARPT'
elif 'CAPITAL' in all_files[0]:
    cube_name = 'CAPITAL'
elif 'CAPT' in all_files[0]:
    cube_name = 'CAPTRPT'
elif 'STAFF' in all_files[0]:
    cube_name = 'ALLSTAFF'
else:
    pass

# Concatente all of the dataframes
essbase = pd.concat(df for df in df_list)


In [106]:
print(all_files)

['C:\\Users\\e79230\\OneDrive - Southwest Airlines\\Documents\\MyJupyterFiles\\\\data\\essbase\\Export_SWA_RPT_CY-NY5_All_Lev0.txt']


In [107]:
# Ensure the exact format of the month names
def month_abbrv(m):
    if 'jan' in m.lower():
        return 'Jan'
    elif 'feb' in m.lower():
        return 'Feb'
    elif 'mar' in m.lower():
        return 'Mar'
    elif 'apr' in m.lower():
        return 'Apr'
    elif 'may' in m.lower():
        return 'May'
    elif 'jun' in m.lower():
        return 'Jun'
    elif 'jul' in m.lower():
        return 'Jul'
    elif 'aug' in m.lower():
        return 'Aug'
    elif 'sep' in m.lower():
        return 'Sep'
    elif 'oct' in m.lower():
        return 'Oct'
    elif 'nov' in m.lower():
        return 'Nov'
    elif 'dec' in m.lower():
        return 'Dec'
    else:
        return 'BegBalance'

In [108]:
# Ensure the format of the dimension header labels used during comparisons
def dimension_cleaner(d):
    if 'VERSION' in d:
        return 'VER'
    elif 'SCENARIO'in d:
        return 'SCEN'
    elif 'YEAR' in d:
        return 'YEAR'    

In [109]:
essbase.shape

(2008022, 24)

In [110]:
#Optional: read a text file to get the list of all capacity accounts
# Even though the text file contains only one column, read_csv still creates a DataFrame

#accounts_all_file = pd.read_csv(r'./data/Capacity_Accounts_ALL.txt',header=None)  # Let pandas determine the delimter (/n)
accounts_all_file = pd.read_csv(path + '\\data\\Capacity_Accounts_ALL.txt',header=None)  # Let pandas determine the delimter (/n)

In [111]:
#Optional: read a text file to get the list of capacity accounts Greg uses in his forecasts
# Even though the text file contains only one column, read_csv still creates a DataFrame
accounts_greg_file = pd.read_csv(path + '\\data\\FINSTMT_UniqueAccounts_CurrCapacity2Working.txt',header=None)  # Let pandas determine the delimter (/n)

In [112]:
# Manual alternative to the text file - these are all of the actively used capacity accounts
accounts_manual = ['FT:0010','FT:0023','FT:0030','FT:0032','FT:0040','FT:0050','FT:0102','FT:0110','FT:0114','FT:0115', \
'FX:00001','FX:00002','FX:00003','FX:00004','FX:00005','FX:00006','FX:00007', 'FX:00008','ST:0117','ST:0270', \
'ST:0272','ST:0273','ST:0320','ST:0321', 'FT:0031','FT:0055','FT:0136','FT:0137','FT:0138','FT:0148','FT:0149','FT:0150', \
'FT:0156','FT:0157','FT:0185','FT:0186', 'FT:0187','FT:0188','FT:0189','FT:0196','FT:0197','FT:0198','ST:0209','ST:0275', \
'ST:0038','ST:0324','ST:0325']

In [113]:
# These accounts are used for station-level capacity forecasts
accounts_stationlevel = ['FT:0023','FT:0030','FT:0110','FT:0115']  

In [114]:
accounts = accounts_all_file

In [115]:
# Optional: read a text file to get a list of PIDs to filter on
pids = pd.read_csv(path + '\\data\\UniquePIDs.txt',header=None)  # Let pandas determine the delimter (/n)
#pids = pids.values.tolist()

In [116]:
# Optional: read a text file to get the list of cost centers to filter on
cost_centers = pd.read_csv(path + '\\data\\Capacity_CostCenters_Dec2018.txt',header=None)  # Let pandas determine the delimter (/n)

In [117]:
flash_versions = ['VFC','VFP','VYC','VYP','VEC','VBC','VTGT','VSX','VSX2','VSX3','VSX4','VSX5','VSX6','VSX7','VSX8','VSX9']

In [118]:
# Set variables

col_types_text = {'ET':'category','PC':'category','CO':'category','TYPE':'category','IO':'category','CC':'category','YEAR':'category','VER':'category','SCEN':'category','ACCT':'category','FILENAME':'category'}
col_types_numeric = {'BegBalance':'float32','Jan':'float32','Feb':'float32','Mar':'float32','Apr':'float32','May':'float32','Jun':'float32','Jul':'float32','Aug':'float32','Sep':'float32','Oct':'float32','Nov':'float32','Dec':'float32'}

col_types_text_allstaff = {'YEAR':'category','SCEN':'category','VER':'category','CC':'category','TENURE':'category','STATUS':'category','MEASURE':'category','ACCT':'category','FILENAME':'category','PERIOD':'category'}
col_types_numeric_allstaff = {'DATA':'int32'}

if cube_name in ['SWARPT','FINSTMT']:
    melt_id_vars = ['ET','PC','CO','TYPE','IO','CC','YEAR','VER','SCEN','ACCT','FILENAME']
    melt_value_vars = ['BegBalance','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
elif cube_name == 'CAPTRPT':
    melt_id_vars = ['PID','PC','ET','TYPE','YEAR','SCEN','CO','VER','CC','ACCT','FILENAME']
    melt_value_vars = ['BegBalance','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
elif cube_name == 'CAPITAL':
    melt_id_vars = ['PID','PC','ET','YEAR','SCEN','CO','VER','CC','ACCT','FILENAME']
    melt_value_vars = ['BegBalance','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

sort_order = ['YEAR','SCEN','VER','CC','PERIOD','PC','IO','ACCT']
sort_direction = [1,1,1,1,1,1,1,1]

if cube_name in ['SWARPT','FINSTMT']:
    sort_order2 = ['YEAR','SCEN','VER','CC','PERIOD','PC','IO','ACCT']
    sort_direction2 = [1,1,1,1,1,1,1,1]
elif cube_name in ['CAPTRPT','CAPITAL']:
    sort_order2 = ['YEAR','SCEN','CC','PERIOD','PC','PID','ACCT']
    sort_direction2 = [1,1,1,1,1,1,1]
    
column_headers_finstmt = ['ET','PC','CO','TYPE','IO','CC','YEAR','VER','SCEN','ACCT','BegBalance','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','FILENAME']

column_headers_swarpt = ['IO','PC','ET','YEAR','CO','TYPE','CC','VER','SCEN','ACCT','BegBalance','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','FILENAME']

column_headers_capital = ['ET','PC','CO','YEAR','CC','PID','VER','SCEN','ACCT','BegBalance','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','FILENAME']

column_headers_captrpt = ['PID','PC','ET','TYPE','YEAR','SCEN','CO','VER','CC','ACCT','BegBalance','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','CashFlow_Period','FILENAME']

column_headers_allstaff = ['YEAR','SCEN','VER','CC','TENURE','STATUS','MEASURE','ACCT', \
                           'BegBalance', 'Week 01', 'Week 02','Week 03','Week 04','Week 05','Week 06','Week 07','Week 08','Week 09',\
                           'Week 10','Week 11', 'Week 12','Week 13','Week 14','Week 15','Week 16','Week 17','Week 18','Week 19',\
                           'Week 20','Week 21', 'Week 22','Week 23','Week 24','Week 25','Week 26','Week 27','Week 28','Week 29',\
                           'Week 30','Week 31', 'Week 32','Week 33','Week 34','Week 35','Week 36','Week 37','Week 38','Week 39',\
                           'Week 40','Week 41', 'Week 42','Week 43','Week 44','Week 45','Week 46','Week 47','Week 48','Week 49',\
                           'Week 50','Week 51', 'Week 52','Week 53','YearTotal','Weeks','Period','FILENAME']


In [119]:
essbase.sample(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,FILENAME
895881,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:16032,VSX,Flash_Base,GL:5646858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
1129799,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14568,VSX4,Flash_Base,GL:5550200,NaN,NaN,NaN,NaN,1.449949e+03,2.231926e+03,1.719131e+03,1.719131e+03,1.719131e+03,1.719131e+03,1.719131e+03,1719.131199,1719.131199,Users\e79230\OneDrive - Southwest Airlines\Doc...
555034,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14606,VYC,Flash_Base,GL:5550500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
1892842,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14637,Locked,Forecast,FT:0025,NaN,6269240.00,6217742.00,9199653.00,8.464774e+06,8.571506e+06,1.331638e+07,1.403060e+07,1.263724e+07,1.243621e+07,1.295796e+07,NaN,NaN,Users\e79230\OneDrive - Southwest Airlines\Doc...
1368822,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14510,VSX7,Flash_Base,HC:32614,NaN,2.00,2.00,2.00,2.000000e+00,3.000000e+00,3.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,Users\e79230\OneDrive - Southwest Airlines\Doc...
915161,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:12310,VSX2,Flash_Base,GL:4378100,NaN,7749.21,8440.59,7747.10,7.908250e+03,7.337620e+03,6.532610e+03,6.877258e+03,6.725174e+03,6.602279e+03,6.566725e+03,6476.141632,6637.288608,Users\e79230\OneDrive - Southwest Airlines\Doc...
834188,IO:382285,PC:1000,ET:None,FY23,CO:9001,Amount,CC:37000,VSX,Flash_Base,GL:5669600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
1542558,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14565,VSX9,Flash_Base,GL:4311000,NaN,39496.50,30088.48,37120.79,2.821330e+04,2.940261e+04,2.863845e+04,3.003754e+04,2.968784e+04,2.724967e+04,2.869459e+04,27497.771893,27497.771893,Users\e79230\OneDrive - Southwest Airlines\Doc...
90140,ZITX,PC:1000,ET:None,FY30,CO:9001,Amount,CC:24057,Final,Actual,GL:5540527,NaN,NaN,NaN,NaN,NaN,7.500000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Users\e79230\OneDrive - Southwest Airlines\Doc...
930898,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:39017,VSX2,Flash_Base,GL:5824000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...


In [120]:
# Write the column headers
if cube_name == 'FINSTMT':
    essbase.columns = column_headers_finstmt
elif cube_name == 'SWARPT':
    essbase.columns = column_headers_swarpt
elif cube_name == 'CAPITAL':
    essbase.columns = column_headers_capital
elif cube_name == 'CAPTRPT':
    essbase.columns = column_headers_captrpt
elif cube_name == 'ALLSTAFF':
    essbase.columns = column_headers_allstaff
else:
    pass

In [121]:
essbase.sample(10)

,IO,PC,ET,YEAR,CO,TYPE,CC,VER,SCEN,ACCT,BegBalance,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,FILENAME
629275,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14502,VYP,Flash_Base,HC:13110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
73983,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14563,Final,Actual,GL:5573400,NaN,7409.220000,3420.53,6555.530000,3756.69,6415.420000,8159.250000,7509.560000,8760.530000,4003.610000,6691.170000,8999.330000,NaN,Users\e79230\OneDrive - Southwest Airlines\Doc...
528724,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:15648,VYC,Flash_Base,GL:5540550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
1295123,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:16013,VSX6,Flash_Base,GL:5539900,NaN,NaN,NaN,662.210000,NaN,105.431678,105.431678,105.431678,105.431678,105.431678,105.431678,105.431678,105.431678,Users\e79230\OneDrive - Southwest Airlines\Doc...
1135494,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:16016,VSX4,Flash_Base,GL:4264000,NaN,NaN,NaN,NaN,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
147186,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:16016,Final,Budget,GL:5550225,NaN,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
995579,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14706,VSX2,Flash_Base,GL:5793300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
882161,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14561,VSX,Flash_Base,GL:4261100,NaN,599322.750000,498621.40,604458.570000,575861.65,618403.910000,605263.660000,608296.370000,NaN,NaN,NaN,NaN,NaN,Users\e79230\OneDrive - Southwest Airlines\Doc...
1376554,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:16035,VSX7,Flash_Base,GL:4311000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
398766,IO:None,PC:1000,ET:None,FY25,CO:9001,Amount,CC:39001,VFP,Flash_Base,GL:4378200,NaN,2708.864528,NaN,74.192841,NaN,147.435283,296.560166,222.895324,220.202524,73.084041,73.084041,NaN,2119.437201,Users\e79230\OneDrive - Southwest Airlines\Doc...


In [122]:
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    #Add an "ALL" selection at the top of the list for all dimensions other than Scenario and Version
    if (not 'Forecast' in unique) and (not 'Working' in unique): 
        unique.insert(0, 'ALL')
    return unique

output = widgets.Output()

# Create the selection lists and radio buttons
# For the lists, make the default selection "ALL" for everything except Scenario and Version
multiselect_year = widgets.SelectMultiple(options = unique_sorted_values_plus_ALL(essbase['YEAR']),description='Year',value=['ALL'])
multiselect_scenario = widgets.SelectMultiple(options = unique_sorted_values_plus_ALL(essbase['SCEN']),description='Scenario')
multiselect_version = widgets.SelectMultiple(options = unique_sorted_values_plus_ALL(essbase['VER']),description='Version')
multiselect_acct = widgets.SelectMultiple(options = unique_sorted_values_plus_ALL(essbase['ACCT']),description='Account',value=['ALL'])
multiselect_cc = widgets.SelectMultiple(options = unique_sorted_values_plus_ALL(essbase['CC']),description='Cost Center',value=['ALL'])

if not cube_name == 'ALLSTAFF' and not cube_name == 'CAPITAL':
    multiselect_type = widgets.SelectMultiple(options = unique_sorted_values_plus_ALL(essbase['TYPE']),description='Type',value=['ALL'])

if cube_name == 'ALLSTAFF':
    pass
elif cube_name in ['CAPITAL','CAPTRPT']:
    multiselect_io = widgets.SelectMultiple(options = unique_sorted_values_plus_ALL(essbase['PID']),description='PID',value=['ALL'])
else:
    multiselect_io = widgets.SelectMultiple(options = unique_sorted_values_plus_ALL(essbase['IO']),description='Internal Order',value=['ALL'])

capacity_accounts = widgets.Checkbox(value=False,description='Use Capacity Accounts Only',disabled=False,indent=True)
predefined_cost_centers = widgets.Checkbox(value=False,description='Use Predefined Cost Centers Only',disabled=False,indent=True)
predefined_pids = widgets.Checkbox(value=False,description='Use Predefined PIDs Only',disabled=False,indent=True)

filter_fcst_months = widgets.Checkbox(value=False,description='Filter By Forecast Months',disabled=False,indent=True)
select_fcstmonth1 = widgets.Select(options = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],description='First Fcst Month')

select_comps = widgets.Select(options = ['Before/After','Years','Scenarios','Versions','Scenario/Version Combos'],value=None,description='Compare What?')
select_scenario_version_combos = widgets.Select(options = ['Actual/Final vs Working/Forecast'],description='Scen/Ver',value=None)

radio_compare = widgets.RadioButtons(options=['Yes','No'],description='Compare Anything?',value='No')

radio_diffsonly = widgets.RadioButtons(options=['Yes','No'],description='Show Only Rows With Differences?',value='Yes')
radio_onezero = widgets.RadioButtons(options=['Yes','No'],description='Show One-Zero Rows?',value='No')
radio_nozeros = widgets.RadioButtons(options=['Yes','No'],description='Show No-Zero Rows?',value='No')

radio_sameonly = widgets.RadioButtons(options=['Yes','No'],description='Show Only Rows That Are Equal?',value='No')
radio_dropempty = widgets.RadioButtons(options=['Yes','No'],description='Drop Empty Rows?',value='Yes')

create_load_file = widgets.Checkbox(value=False,description='Create Load File?',disabled=False,indent=True)

suppress_file_name = widgets.Checkbox(value=False,description='Suppress FILENAME Field?',disabled=False,indent=True)

only_concatenate_files = widgets.Checkbox(value=False,description='Only Concatenate Files?',disabled=False,indent=True)


# Not currently being used
def common_filtering(year, scenario, version):
    output.clear_output()
    if (year == ALL) & (scenario == ALL) & (version == ALL):
        common_filter = essbase
    elif (year == ALL):
        common_filter = essbase[(essbase['SCEN'] == scenario) & (essbase['VER'] == version)]
    elif (scenario == ALL):
        common_filter = essbase[(essbase['YEAR'] == year) & (essbase['VER'] == version)]
    elif (version == ALL):
        common_filter = essbase[(essbase['YEAR'] == year) & (essbase['SCEN'] == scenario)]
    else:
        common_filter = essbase[(essbase['YEAR'] == year) & (essbase['SCEN'] == scenario) & (essbase['VER'] == version)]
    
    with output:
        display(common_filter)

# Not currently being used
def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_scenario.value, dropdown_version.value)
    
# Not currently being used           
def dropdown_scenario_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, dropdown_version.value)

# Not currently being used
def dropdown_version_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_scenario.value, change.new)
            
# Display the widgets horiontally, in groups
input_widgets1 = widgets.HBox([multiselect_year, multiselect_scenario, multiselect_version])
display(input_widgets1)

input_widgets2 = widgets.HBox([multiselect_acct, multiselect_cc])
display(input_widgets2)

if not cube_name == 'ALLSTAFF':
    input_widgets2b = widgets.HBox([multiselect_io])
    display(input_widgets2b)
    
    display(capacity_accounts)
    display(predefined_cost_centers)
    display(predefined_pids)

    if not cube_name == 'CAPITAL':
        input_widgets3 = widgets.HBox([multiselect_type])
        display(input_widgets3)

    input_widgets4 = widgets.HBox([filter_fcst_months, select_fcstmonth1])
    display(input_widgets4)

input_widgets5 = widgets.HBox([radio_compare, select_comps, select_scenario_version_combos])
display(input_widgets5)

input_widgets6 = widgets.HBox([radio_diffsonly, radio_onezero, radio_nozeros])
display(input_widgets6)

input_widgets7 = widgets.HBox([radio_sameonly, radio_dropempty])
display(input_widgets7)

input_widgets8 = widgets.HBox([create_load_file])
display(input_widgets8)

input_widgets9 = widgets.HBox([suppress_file_name])
display(input_widgets9)

#only_concatenate_files
input_widgets10 = widgets.HBox([only_concatenate_files])
display(input_widgets10)



Checkbox(value=False, description='Use Capacity Accounts Only')

Checkbox(value=False, description='Use Predefined Cost Centers Only')

Checkbox(value=False, description='Use Predefined PIDs Only')

In [28]:
# INSTRUCTIONS:
# 1. Run all cells above this one (from the menu, select Cell ==> Run All Above)
# 2. Make your selections in the widgets
#    WARNING: For comparisons, verify the values of all radio button options
# 3. Run the next cell (by itself) to confirm your selections
# 4. Re-select that cell, then run all cells below it (Cell ==> Run All Below)

In [123]:
print('Years: ' + str(list(multiselect_year.value)))
print('Scenario: ' + str(list(multiselect_scenario.value)))
print('Version: ' + str(list(multiselect_version.value)))
print('Account: ' + str(list(multiselect_acct.value)) + '  Use Capacity Accounts Only?: ' + str(capacity_accounts.value))
print('Cost Center: ' + str(list(multiselect_cc.value)))

#if not cube_name == 'ALLSTAFF':
if not cube_name in ['ALLSTAFF','CAPITAL']:
    #print('Internal Order/PID: ' + str(list(multiselect_io.value)))
    print('Internal Order/PID: ' + str(list(multiselect_io.value)) + '  Use Predefined IOs/PIDs Only?: ' + str(predefined_pids.value))
    print('Type: ' + str(list(multiselect_type.value)))

print('Compare Anything?: ' + radio_compare.value)
if radio_compare.value == 'Yes':
    print('Compare What?: ' + select_comps.value)
    if select_comps.value == 'Scenario/Version Combos':
        print('Scenario/Version Combos: ' + select_scenario_version_combos.value)
print('Show Only Rows With Differences?: ' + radio_diffsonly.value)
print('Show Only Rows With One Zero?: ' + radio_onezero.value)
print('Show Only Rows With No Zeros?: ' + radio_nozeros.value)
print('Show Only Rows With Equal Values?: ' + radio_sameonly.value)
print('Drop Rows With Two Zeros?: ' + radio_dropempty.value)
print('Filter By Forecast Months?: ' + str(filter_fcst_months.value))
print('First Forecast Month: ' + select_fcstmonth1.value)
print('Create Load File?: ' + str(create_load_file.value))
print('Suppress FileName Field?: ' + str(suppress_file_name.value))
print('Only concatenate files?: ' + str(only_concatenate_files.value))

Years: ['FY24']
Scenario: ['Flash_Base', 'Flash_Eco', 'Flash_GAAP', 'Flash_NonGAAP']
Version: ['VFP']
Account: ['ALL']  Use Capacity Accounts Only?: False
Cost Center: ['ALL']
Internal Order/PID: ['ALL']  Use Predefined IOs/PIDs Only?: False
Type: ['ALL']
Compare Anything?: No
Show Only Rows With Differences?: Yes
Show Only Rows With One Zero?: No
Show Only Rows With No Zeros?: No
Show Only Rows With Equal Values?: No
Drop Rows With Two Zeros?: Yes
Filter By Forecast Months?: False
First Forecast Month: Jan
Create Load File?: True
Suppress FileName Field?: False
Only concatenate files?: False


In [124]:
# To recover Working Forecast data for an entire year (with accuracy to 10+ decimals)

# Source file: any BOD file
# Manually load the output file into FIN_STMT with the _MXL_Batch_Load load rule

##### Widget Selections #####
# Years: ['FY20']
# Scenario: ['Forecast']
# Version: ['Working']
# Account: ['ALL']  Use Capacity Accounts Only?: False
# Cost Center: ['ALL']
# Internal Order/PID: ['ALL']
# Compare Anything?: No
# Compare What?: Before/After
# Show Only Rows With Differences?: Yes
# Show Only Rows With One Zero?: No
# Show Only Rows With No Zeros?: No
# Show Only Rows With Equal Values?: No
# Drop Rows With Two Zeros?: Yes

In [125]:
# To recover and/or compare a Flash version in SWA_RPT, filter on the version and year only, and not ANY scenarios
# NOTE: This will return Flash_GAAP and Flash_Eco rows, as well as Flash_NonGAAP

##### Widget Selections #####
# Years: ['FY20']
# Scenario: ['ALL']
# Version: ['VFC']
# Account: ['ALL']  Use Capacity Accounts Only?: False
# Cost Center: ['ALL']
# Internal Order/PID: ['ALL']
# Compare Anything?: No
# Compare What?: Before/After
# Show Only Rows With Differences?: Yes
# Show Only Rows With One Zero?: No
# Show Only Rows With No Zeros?: No
# Show Only Rows With Equal Values?: No
# Drop Rows With Two Zeros?: Yes

In [126]:
# The monthly ExTO actuals adjustments file is now generated by a calc script rather than being created here

In [127]:
# For Working loads, run a Before vs After on Working Forecast
# Must test ALL accounts to see if anything unintended changed

##### Widget Selections #####
# Years: ['FY20', 'FY21']
# Scenario: ['Forecast']
# Version: ['Working']
# Account: ['ALL']  Use Capacity Accounts Only?: False
# Cost Center: ['ALL']
# Internal Order/PID: ['ALL']
# Compare Anything?: Yes
# Compare What?: Before/After
# Show Only Rows With Differences?: Yes
# Show Only Rows With One Zero?: No
# Show Only Rows With No Zeros?: No
# Show Only Rows With Equal Values?: No
# Drop Rows With Two Zeros?: Yes

In [128]:
# For Current Capacity *loads*, run a Before vs After on Working and Current Capacity
# Must test ALL accounts to see if anything unintended changed

##### Widget Selections #####
# Years: ['FY20', 'FY21']
# Scenario: ['Forecast']
# Version: ['Current Capacity', 'Working']
# Account: ['ALL']  Use Capacity Accounts Only?: False
# Cost Center: ['ALL']
# Internal Order/PID: ['ALL']
# Compare Anything?: Yes
# Compare What?: Before/After
# Show Only Rows With Differences?: Yes
# Show Only Rows With One Zero?: No
# Show Only Rows With No Zeros?: No
# Show Only Rows With Equal Values?: No
# Drop Rows With Two Zeros?: Yes

In [129]:
# For Current Capacity *copies to Working*, run Before vs After AND Working vs Current Capacity comparisons

# For the Working vs Current Capacity comps the only file needed is the "After Load" export
# Enable the "Use Capacity Accounts Only" option

# The only accounts that should be in the output file are: FT:0020, FT:0101, FT:0164, FT:0165, FT:0194, and FT:0195
# ALL OTHER CAPACITY ACCOUNTS APPEARING IN THE OUTPUT MUST BE FIXED TO MAKE THEM EQUAL CURRENT CAPACITY

##### Widget Selections #####
# Years: ['FY21']
# Scenario: ['Forecast']
# Version: ['Current Capacity', 'Working']
# Account: ['ALL']  Use Capacity Accounts Only?: True
# Cost Center: ['ALL']
# Internal Order/PID: ['ALL']
# Compare Anything?: Yes
# Compare What?: Versions
# Show Only Rows With Differences?: Yes
# Show Only Rows With One Zero?: No
# Show Only Rows With No Zeros?: No
# Show Only Rows With Equal Values?: No
# Drop Rows With Two Zeros?: Yes

In [130]:
# For AllStaff only, melt the time periods
# Melt all of the months into a new column named PERIOD; parameter values defined in the variables cell above
if cube_name == 'ALLSTAFF':
    essbase = pd.melt(essbase, id_vars=['YEAR','SCEN','VER','CC','TENURE','STATUS','MEASURE','ACCT','FILENAME'], var_name='PERIOD',value_name='DATA')
    essbase = essbase.fillna(value={'DATA':0})
    essbase.head()

In [131]:
essbase.sample(10)

,IO,PC,ET,YEAR,CO,TYPE,CC,VER,SCEN,ACCT,BegBalance,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,FILENAME
1716267,IO:None,PC:1000,ET:None,FY24,CO:9001,Amount,CC:14511,Working,Forecast,GL:4378200,NaN,2792.212467,1923.766073,979.518836,838.127103,529.230713,525.761150,490.568186,427.496121,397.349586,365.955874,330.657580,2274.602607,Users\e79230\OneDrive - Southwest Airlines\Doc...
1248936,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:34009,VSX6,Flash_Base,GL:5669200,NaN,NaN,NaN,NaN,NaN,41.666667,41.666667,41.666667,41.666667,41.666667,41.666667,41.666667,41.666667,Users\e79230\OneDrive - Southwest Airlines\Doc...
1893279,IO:None,PC:1000,ET:None,FY24,CO:9001,Amount,CC:16024,Locked,Forecast,GL:5560600,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
635928,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14580,VYP,Flash_Base,GL:5840600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
1654687,IO:None,PC:1000,ET:None,FY24,CO:9001,Amount,CC:15070,Working,Forecast,GL:5540475,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
178253,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:12213,Final,AOP,GL:4220000,NaN,9386.000000,9386.000000,9667.580000,9667.580000,9668.000000,9668.000000,9668.000000,9668.000000,9668.000000,9668.000000,9668.000000,9668.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
1411666,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:17588,VSX8,Flash_Base,ST:1500,NaN,93.000000,115.000000,157.000000,120.000000,153.000000,141.000000,115.000000,125.000000,NaN,NaN,NaN,NaN,Users\e79230\OneDrive - Southwest Airlines\Doc...
899254,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14026,VSX,Flash_Base,GL:5436200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,Users\e79230\OneDrive - Southwest Airlines\Doc...
1941686,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:27003,Final,Forecast,GL:4311000,NaN,28776.556400,30247.759425,31579.795325,31579.795325,31579.795325,31579.795325,32326.715525,32326.715525,32326.715525,32326.715525,32326.715525,NaN,Users\e79230\OneDrive - Southwest Airlines\Doc...
1597295,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:18001,Working,Forecast,GL:5601300,NaN,NaN,NaN,69.940000,NaN,NaN,42.850000,NaN,NaN,NaN,NaN,NaN,NaN,Users\e79230\OneDrive - Southwest Airlines\Doc...


In [132]:
if only_concatenate_files.value == True:
    # Drop the FILENAME column
    essbase = essbase.drop('FILENAME',axis='columns')
    essbase.to_csv (r'C:\zDailyWork\\' + cube_name + '_' + current_date + '.txt', index = False, header = False, sep='|')
    raise SystemExit("Stop right here!")  # NOTE: sys.exit(0) does not work properly in Anaconda

In [133]:
# Fill blank numeric cells with zeroes
# Note:  Consider using #Missing as a fill value for recovered data that will be loaded back into a cube
if not cube_name == 'ALLSTAFF':
    essbase = essbase.fillna(value={'BegBalance':0, 'Jan':0, 'Feb':0, 'Mar':0, 'Apr':0, 'May':0, 'Jun':0, 'Jul':0, 'Aug':0, 'Sep':0,'Oct':0, 'Nov':0, 'Dec':0,'CashFlow_Period':0})


In [134]:
essbase.sample(10)

,IO,PC,ET,YEAR,CO,TYPE,CC,VER,SCEN,ACCT,BegBalance,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,FILENAME
502502,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14002,VYC,Flash_Base,GL:5550500,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,Users\e79230\OneDrive - Southwest Airlines\Doc...
1788485,IO:None,PC:1000,ET:None,FY30,CO:9001,Amount,CC:17513,Locked,Forecast,GL:5560600,0.0,0.00,0.00,0.03,0.03,0.03,0.03,0.03,0.030,0.030,0.030,0.030,0.030,Users\e79230\OneDrive - Southwest Airlines\Doc...
819901,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:31011,VSX,Flash_Base,GL:4221000,0.0,28590.97,25662.98,27229.23,27229.22,27229.23,27229.22,27229.22,27433.419,27433.419,27433.419,27433.419,27433.419,Users\e79230\OneDrive - Southwest Airlines\Doc...
861969,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14616,VSX,Flash_Base,GL:5793300,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,Users\e79230\OneDrive - Southwest Airlines\Doc...
902146,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:14990,VSX,Flash_Base,GL:4351000,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,Users\e79230\OneDrive - Southwest Airlines\Doc...
548374,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:16019,VYC,Flash_Base,HC:12110,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,Users\e79230\OneDrive - Southwest Airlines\Doc...
263186,IO:None,PC:1000,ET:None,FY24,CO:9001,Amount,CC:39013,VFC,Flash_Base,GL:5489700,0.0,0.00,0.00,4500.00,0.00,0.00,4500.00,0.00,0.000,4500.000,0.000,0.000,4500.000,Users\e79230\OneDrive - Southwest Airlines\Doc...
185596,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:39002,Final,AOP,GL:5539400,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,Users\e79230\OneDrive - Southwest Airlines\Doc...
105594,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:39003,Final,Budget,GL:5707200,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,Users\e79230\OneDrive - Southwest Airlines\Doc...
1914720,IO:None,PC:1000,ET:None,FY23,CO:9001,Amount,CC:16090,Locked,Forecast,GL:5646858,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,Users\e79230\OneDrive - Southwest Airlines\Doc...


In [135]:
essbase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2008022 entries, 0 to 2008021
Data columns (total 24 columns):
 #   Column      Dtype  
---  ------      -----  
 0   IO          object 
 1   PC          object 
 2   ET          object 
 3   YEAR        object 
 4   CO          object 
 5   TYPE        object 
 6   CC          object 
 7   VER         object 
 8   SCEN        object 
 9   ACCT        object 
 10  BegBalance  float64
 11  Jan         float64
 12  Feb         float64
 13  Mar         float64
 14  Apr         float64
 15  May         float64
 16  Jun         float64
 17  Jul         float64
 18  Aug         float64
 19  Sep         float64
 20  Oct         float64
 21  Nov         float64
 22  Dec         float64
 23  FILENAME    object 
dtypes: float64(13), object(11)
memory usage: 367.7+ MB


In [136]:
# Optional: Convert data types to conserve memory

## Downcasting 'object' types to 'category' is an important conversion
## With two full FIN_STMT extracts in the dataframe, this reduces memory usage from 185 MB to 130
#if not cube_name == "ALLSTAFF":
#    essbase = essbase.astype(col_types_text)
#else:
#    essbase.fillna(value={'DATA':0})
#    essbase = essbase.astype(col_types_text_allstaff)
#    essbase = essbase.astype(col_types_numeric_allstaff)
    
    
# HOWEVER, categorical data may causes complications when melting and pivoting data. Therefore, it may not be not worth it.

# WARNING: When using categoricals while creating a pivot table, the 'observed' parameter in pivot_table must be set to True 
# If it's set to False (the default) a cartesian product of the category combinations will be created in the pivot table

# Can't convert to float32 in data sets containing ASMs because of the float32 value limit of 2,147,483,647 
#essbase = essbase.astype(col_types_numeric)

#essbase.info()

In [137]:
# This worked on 10/16/20
#accounts_original = pd.DataFrame(accounts_manual)
#print (len(accounts_original.index))
#accounts_expanded = accounts_file
#print (len(accounts_expanded.index))
#accounts_concat = pd.concat([accounts_original, accounts_expanded])
#print (len(accounts_concat.index))
#accounts_unknown = accounts_concat.drop_duplicates(keep=False)
#print (len(accounts_unknown.index))
#accounts = accounts_unknown

In [138]:
#accounts_unknown

In [139]:
# Apply the widget selections

if not 'ALL' in list(multiselect_year.value):
    essbase = essbase[essbase['YEAR'].isin(list(multiselect_year.value))]

if not 'ALL' in list(multiselect_scenario.value):
    essbase = essbase[essbase['SCEN'].isin(list(multiselect_scenario.value))]

if not 'ALL' in list(multiselect_version.value):
    essbase = essbase[essbase['VER'].isin(list(multiselect_version.value))]

if capacity_accounts.value == True:
    essbase = essbase[essbase['ACCT'].isin(accounts[0])]
else:
    if not 'ALL' in list(multiselect_acct.value):
        essbase = essbase[essbase['ACCT'].isin(list(multiselect_acct.value))]

if not 'ALL' in list(multiselect_cc.value):
    essbase = essbase[essbase['CC'].isin(list(multiselect_cc.value))]

if cube_name != 'ALLSTAFF':
    if predefined_pids.value == True:
        if cube_name in ['CAPITAL','CAPTRPT']:
            essbase = essbase[essbase['PID'].isin(pids[0])]
    elif not 'ALL' in list(multiselect_io.value):
        if cube_name in ['CAPITAL','CAPTRPT']:
            essbase = essbase[essbase['PID'].isin(list(multiselect_io.value))]
        else:
            essbase = essbase[essbase['IO'].isin(list(multiselect_io.value))]
    
    if cube_name !='CAPITAL':
        if not 'ALL' in list(multiselect_type.value):
        #if cube_name !='CAPITAL':
            essbase = essbase[essbase['TYPE'].isin(list(multiselect_type.value))]
            

In [140]:
essbase.info() # Check memory usage after filtering

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14242 entries, 239697 to 1570761
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   IO          14242 non-null  object 
 1   PC          14242 non-null  object 
 2   ET          14242 non-null  object 
 3   YEAR        14242 non-null  object 
 4   CO          14242 non-null  object 
 5   TYPE        14242 non-null  object 
 6   CC          14242 non-null  object 
 7   VER         14242 non-null  object 
 8   SCEN        14242 non-null  object 
 9   ACCT        14242 non-null  object 
 10  BegBalance  14242 non-null  float64
 11  Jan         14242 non-null  float64
 12  Feb         14242 non-null  float64
 13  Mar         14242 non-null  float64
 14  Apr         14242 non-null  float64
 15  May         14242 non-null  float64
 16  Jun         14242 non-null  float64
 17  Jul         14242 non-null  float64
 18  Aug         14242 non-null  float64
 19  Sep         14242 

In [141]:
# Round all columns to zero decimal places
# WARNING: Do NOT do this if creating a file that will be loaded back into a cube
#essbase = essbase.round(0)

In [142]:
essbase.shape

(14242, 24)

In [143]:
essbase.sample(1)

,IO,PC,ET,YEAR,CO,TYPE,CC,VER,SCEN,ACCT,BegBalance,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,FILENAME
444645,IO:330001,PC:1000,ET:None,FY24,CO:9001,Amount,CC:14643,VFP,Flash_Base,GL:5487500,0.0,50950.463333,50950.463333,50950.463333,50950.463333,50950.463333,50950.463333,55528.8585,55528.8585,55528.8585,55528.8585,55528.8585,55528.8585,Users\e79230\OneDrive - Southwest Airlines\Doc...


In [144]:
#essbase.to_csv (r'C:\zDailyWork\\' + cube_name + '_ActVsFcst1_' + current_date + '.txt', index = False, header=False, sep='|')

In [145]:
# Melt all of the months into a new column named PERIOD; parameter values defined in the variables cell above
if not cube_name == 'ALLSTAFF':
    essbase = pd.melt(essbase, id_vars=melt_id_vars, var_name='PERIOD',value_name='DATA')
    essbase = essbase.fillna(value={'value':0})
    essbase.head()    

,ET,PC,CO,TYPE,IO,CC,YEAR,VER,SCEN,ACCT,FILENAME,PERIOD,DATA
0,ET:None,PC:1000,CO:9001,Adjustment,IO:None,CC:40001,FY24,VFP,Flash_GAAP,GL:4381000,Users\e79230\OneDrive - Southwest Airlines\Doc...,BegBalance,0.0
1,ET:None,PC:1000,CO:9001,Adjustment,IO:None,CC:40001,FY24,VFP,Flash_GAAP,GL:5005210,Users\e79230\OneDrive - Southwest Airlines\Doc...,BegBalance,0.0
2,ET:None,PC:1000,CO:9001,Adjustment,IO:None,CC:40001,FY24,VFP,Flash_GAAP,GL:7858600,Users\e79230\OneDrive - Southwest Airlines\Doc...,BegBalance,0.0
3,ET:None,PC:1000,CO:9001,Adjustment,IO:None,CC:40001,FY24,VFP,Flash_GAAP,GL:7900200,Users\e79230\OneDrive - Southwest Airlines\Doc...,BegBalance,0.0
4,ET:None,PC:1000,CO:9001,Adjustment,IO:None,CC:40001,FY24,VFP,Flash_GAAP,GL:7900400,Users\e79230\OneDrive - Southwest Airlines\Doc...,BegBalance,0.0


In [146]:
essbase.sample(10)

,ET,PC,CO,TYPE,IO,CC,YEAR,VER,SCEN,ACCT,FILENAME,PERIOD,DATA
111464,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:30003,FY24,VFP,Flash_Base,GL:5560700,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jul,0.000000
101967,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:12039,FY24,VFP,Flash_Base,GL:4476902,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jul,8.611658
41583,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:22001,FY24,VFP,Flash_Base,GL:5707100,Users\e79230\OneDrive - Southwest Airlines\Doc...,Feb,206.000000
7632,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:32028,FY24,VFP,Flash_Base,HC:12110,Users\e79230\OneDrive - Southwest Airlines\Doc...,BegBalance,0.000000
136719,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:26000,FY24,VFP,Flash_Base,GL:5600600,Users\e79230\OneDrive - Southwest Airlines\Doc...,Sep,106.332908
1880,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:12065,FY24,VFP,Flash_Base,GL:4382000,Users\e79230\OneDrive - Southwest Airlines\Doc...,BegBalance,0.000000
94306,ET:None,PC:1000,CO:9001,Amount,IO:330001,CC:14607,FY24,VFP,Flash_Base,GL:5487300,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jun,470591.080000
60623,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:17583,FY24,VFP,Flash_Base,GL:4456210,Users\e79230\OneDrive - Southwest Airlines\Doc...,Apr,107.825370
174178,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:34007,FY24,VFP,Flash_Base,GL:4456220,Users\e79230\OneDrive - Southwest Airlines\Doc...,Dec,1130.162922
49830,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:21000,FY24,VFP,Flash_Base,HC:12210,Users\e79230\OneDrive - Southwest Airlines\Doc...,Mar,7.000000


In [147]:
essbase.shape

(185146, 13)

In [148]:
#essbase.to_csv (r'C:\zDailyWork\\' + cube_name + '_ActVsFcst2_' + current_date + '.txt', index = False, header=False, sep='|')

In [149]:
essbase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185146 entries, 0 to 185145
Data columns (total 13 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   ET        185146 non-null  object 
 1   PC        185146 non-null  object 
 2   CO        185146 non-null  object 
 3   TYPE      185146 non-null  object 
 4   IO        185146 non-null  object 
 5   CC        185146 non-null  object 
 6   YEAR      185146 non-null  object 
 7   VER       185146 non-null  object 
 8   SCEN      185146 non-null  object 
 9   ACCT      185146 non-null  object 
 10  FILENAME  185146 non-null  object 
 11  PERIOD    185146 non-null  object 
 12  DATA      185146 non-null  float64
dtypes: float64(1), object(12)
memory usage: 18.4+ MB


In [150]:
def scenario_version_combos(df, sv_combos):
    
    if sv_combos == 'Actual/Final vs Working/Forecast':
        # Remove SCEN and VER from all indexes (they will be pivoted up to the columns)
        if cube_name in ['SWARPT','FINSTMT'] :
            index = ['ET','PC','CO','TYPE','IO','CC','YEAR','ACCT','PERIOD']
        elif cube_name == 'CAPTRPT':
            index = ['ET','PC','CO','TYPE','PID','CC','YEAR','ACCT','PERIOD']
        elif cube_name == 'CAPITAL':
            index = ['ET','PC','CO','PID','CC','YEAR','ACCT','PERIOD']
            
        # Create the Actual/Final dataframe
        cond1 = df['SCEN'] == 'Actual'
        cond2 = df['VER'] == 'Final'
        cond3 = df['DATA'] != 0
        essbase_actuals = df[cond1 & cond2 & cond3]
        essbase_actuals = essbase_actuals.pivot_table(values='DATA', index=index, columns=['SCEN','VER'], observed=True)

        # Create the Working/Forecast dataframe
        cond1 = df['SCEN'] == 'Forecast'
        cond2 = df['VER'] == 'Working'
        cond3 = df['DATA'] != 0
        essbase_forecast = df[cond1 & cond2 & cond3]
        essbase_forecast = essbase_forecast.pivot_table(values='DATA', index=index, columns=['SCEN','VER'], observed=True)
            
        # Join the dataframes
        #essbase_actfcst = essbase_actuals.merge(essbase_forecast, how='inner', on=index)
        essbase_actfcst = essbase_actuals.merge(essbase_forecast, how='outer', on=index)
        
        # Concatenate the two header rows into a single row; na_action suppresses the / when only one header value exists
        essbase_actfcst.columns = essbase_actfcst.columns.map('/'.join, na_action='ignore')  
        
        essbase = essbase_actfcst
        return essbase
        

In [151]:
# Simplify the column names
def rename_col(c):
    print(c)  # c is the column label, which is currently the full name of the extract file
    before_labels = ['BEFORE', 'BOD']  # A list of possible strings embedded within the name of the "Before" extract file
    if any(label in c.upper() for label in before_labels):
        return 'BEFORE'
    else:
        return 'AFTER'   

In [152]:
def comps(df):
    
    #print(df.head())
    
    cond2 = []
   
    df = df.fillna(value={df.columns[0]:0,df.columns[1]:0})
    #df = df.round(0) - do NOT round, in case we need to ever compare rates

    if radio_diffsonly.value == 'Yes':
        show_only_same = 'No'
        
        if radio_onezero.value == 'Yes' and  radio_nozeros.value == 'Yes':
            #cond1 = df[df.columns[0]] != df[df.columns[1]]
            cond1 = abs(df[df.columns[0]] - df[df.columns[1]]) > .000001
            
            # Apply the condition
            df = df[cond1]

        else:
            if radio_onezero.value == 'Yes':
                #cond1 = df[df.columns[0]] != df[df.columns[1]]
                cond1 = abs(df[df.columns[0]] - df[df.columns[1]]) > .000001
                cond2 = df[df.columns[0]] == 0
                cond3 = df[df.columns[1]] == 0
            
                # Apply the condition
                df = df[cond1 & (cond2 | cond3)]
            
            elif radio_nozeros.value == 'Yes':
                #cond1 = df[df.columns[0]] != df[df.columns[1]]
                cond1 = abs(df[df.columns[0]] - df[df.columns[1]]) > .000001
                cond2 = df[df.columns[0]] != 0
                cond3 = df[df.columns[1]] != 0
            
                # Apply the condition
                df = df[cond1 & (cond2 & cond3)]

            else:
                #cond1 = df[df.columns[0]] != df[df.columns[1]]
                cond1 = abs(df[df.columns[0]] - df[df.columns[1]]) > .000001
            
                # Apply the condition
                df = df[cond1]
            
    else:
        if radio_sameonly.value == 'Yes':
            show_only_same = 'Yes'
            if radio_dropempty.value == 'Yes':
                cond1 = df[df.columns[0]] == df[df.columns[1]]
                cond2 = df[df.columns[0]] != 0
            else:
                cond1 = df[df.columns[0]] == df[df.columns[1]]
        else:
            # Show all rows, regardless of values (force a True condition on every row)
            cond1 = df[df.columns[0]] == df[df.columns[0]]
        
        # Apply the condition(s)
        if any(cond2):
            df = df[cond1 & cond2]
        else:
            df = df[cond1]
        
    
    #if show_only_same == 'Yes':
    #    if radio_dropempty.value == 'Yes':
    #        df = df.loc[~(df==0).all(axis=1)]  # In pandas the tilde is the same as the Python !=

    df = df.reset_index()  # IMPORTANT: Get everything back on the same level (i.e., no multi-index in the columns)
    return df
    

In [153]:
# WARNING: Call this function only after all melting and pivoting operations are finished
# Pivoting with any categoricals in the dataframe is VERY slow (even if they remain in the index)

# Create a pandas category called 'months' ('weeks' for ALLSTAFF) and convert PERIOD into that category
# This enables months to be sorted in Jan-Dec order

def set_categoricals(df):
    if cube_name == 'ALLSTAFF':
        weeks = CategoricalDtype(categories=['BegBalance', 'Week 01', 'Week 02','Week 03','Week 04','Week 05','Week 06','Week 07','Week 08','Week 09',\
                 'Week 10','Week 11', 'Week 12','Week 13','Week 14','Week 15','Week 16','Week 17','Week 18','Week 19',\
                 'Week 20','Week 21', 'Week 22','Week 23','Week 24','Week 25','Week 26','Week 27','Week 28','Week 29',\
                 'Week 30','Week 31', 'Week 32','Week 33','Week 34','Week 35','Week 36','Week 37','Week 38','Week 39',\
                 'Week 40','Week 41', 'Week 42','Week 43','Week 44','Week 45','Week 46','Week 47','Week 48','Week 49',\
                 'Week 50','Week 51', 'Week 52','Week 53','YearTotal','Weeks','Period'],ordered=True)
        df['PERIOD'] = df['PERIOD'].astype(weeks)
    else:
        if cube_name in ['CAPTRPT','CAPITAL']:
            months = CategoricalDtype(categories=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','CashFlow_Period','BegBalance'],ordered=True)
        else:
            months = CategoricalDtype(categories=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','BegBalance'],ordered=True)
    
        df['PERIOD'] = df['PERIOD'].astype(months)
        
    return df

In [154]:
if radio_compare.value == 'Yes':
    
    if select_comps.value == 'Before/After':
        
        if cube_name in ['SWARPT','FINSTMT'] :
            index = ['ET','PC','CO','TYPE','IO','CC','YEAR','VER','SCEN','ACCT','PERIOD']
            essbase = essbase.pivot_table(values='DATA', index=index, columns='FILENAME', observed=True)
        elif cube_name == 'CAPTRPT':
            index = ['ET','PC','CO','TYPE','PID','CC','YEAR','VER','SCEN','ACCT','PERIOD']
            essbase = essbase.pivot_table(values='DATA', index=index, columns='FILENAME', observed=True)
        elif cube_name == 'CAPITAL':
            index = ['ET','PC','CO','PID','CC','YEAR','VER','SCEN','ACCT','PERIOD']
            essbase = essbase.pivot_table(values='DATA', index=index, columns='FILENAME', observed=True)
        elif cube_name == 'ALLSTAFF':
            essbase = essbase.pivot_table(values='DATA', index=['YEAR','SCEN','VER','CC','TENURE','STATUS','MEASURE','ACCT','PERIOD'], columns='FILENAME', observed=True)

        #print(essbase.head())
        #essbase.sample(50)
        
        # The extract filenames are currently the column labels
        # Call the rename_col function above to replace them with 'Before' and 'After'
        essbase = essbase.rename(columns={essbase.columns[0]:rename_col(essbase.columns[0]), essbase.columns[1]:rename_col(essbase.columns[1])})
        essbase = essbase.rename_axis([None], axis='columns')
        essbase = essbase.sort_index(axis=1,ascending=False)
        
        essbase.sample(50)

    elif select_comps.value == 'Scenario/Version Combos':
        
        essbase = scenario_version_combos(essbase, select_scenario_version_combos.value)
        #essbase.to_csv (r'C:\zDailyWork\\' + cube_name + '_ActVsFcst3_' + current_date + '.txt', index = False, header=False, sep='|')

            
    else:
        
        if cube_name in ['SWARPT','FINSTMT']:
            if select_comps.value in ['Versions']:
                index=['ET','PC','CO','TYPE','IO','CC','YEAR','SCEN','ACCT','FILENAME','PERIOD']
            elif select_comps.value in ['Scenarios']:
                index=['ET','PC','CO','TYPE','IO','CC','YEAR','VER','ACCT','FILENAME','PERIOD']
            elif select_comps.value in ['Years']:
                index=['ET','PC','CO','TYPE','IO','CC','VER','SCEN','ACCT','FILENAME','PERIOD']
            
            dimension = dimension_cleaner(select_comps.value.upper())
            essbase = essbase.pivot_table(values='DATA', index=index, columns=dimension, observed=True)
            
        elif cube_name in ['CAPTRPT','CAPITAL']:
            if select_comps.value in ['Versions']:
                if cube_name == 'CAPTRPT':
                    index=['ET','PC','CO','TYPE','PID','CC','YEAR','SCEN','ACCT','FILENAME','PERIOD']
                else:
                    index=['ET','PC','CO','PID','CC','YEAR','SCEN','ACCT','FILENAME','PERIOD']
            elif select_comps.value in ['Scenarios']:
                if cube_name == 'CAPTRPT':
                    index=['ET','PC','CO','TYPE','PID','CC','YEAR','VER','ACCT','FILENAME','PERIOD']
                else:
                    index=['ET','PC','CO','PID','CC','YEAR','VER','ACCT','FILENAME','PERIOD']
            elif select_comps.value in ['Years']:
                if cube_name == 'CAPTRPT':
                    index=['ET','PC','CO','TYPE','PID','CC','VER','SCEN','ACCT','FILENAME','PERIOD']
                else:
                    index=['ET','PC','CO','PID','CC','VER','SCEN','ACCT','FILENAME','PERIOD']
            
            dimension = dimension_cleaner(select_comps.value.upper())
            essbase = essbase.pivot_table(values='DATA', index=index, columns=dimension, observed=True)

        elif cube_name in ['ALLSTAFF']:
            if select_comps.value in ['Versions']:
                pass
            elif select_comps.value in ['Scenarios']:
                pass
            elif select_comps.value in ['Years']:
                index =['SCEN','VER','CC','TENURE','STATUS','MEASURE','ACCT','FILENAME','PERIOD']
                #index=['ET','PC','CO','TYPE','PID','CC','VER','SCEN','ACCT','FILENAME','PERIOD']
                
            dimension = dimension_cleaner(select_comps.value.upper())
            essbase = essbase.pivot_table(values='DATA', index=index, columns=dimension, observed=True)
            
    #NOTE: Comps happen in this custom function,so make any necessary logic tweaks there
    essbase = comps(essbase)
    
else:
    cond1 = essbase['DATA'] != 0 
    essbase = essbase[cond1]


In [155]:
essbase.head()

,ET,PC,CO,TYPE,IO,CC,YEAR,VER,SCEN,ACCT,FILENAME,PERIOD,DATA
14243,ET:None,PC:1000,CO:9001,Adjustment,IO:None,CC:40001,FY24,VFP,Flash_GAAP,GL:5005210,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,-1.130486e+07
14244,ET:None,PC:1000,CO:9001,Adjustment,IO:None,CC:40001,FY24,VFP,Flash_GAAP,GL:7858600,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,3.051000e+06
14245,ET:None,PC:1000,CO:9001,Adjustment,IO:None,CC:40001,FY24,VFP,Flash_GAAP,GL:7900200,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,-5.033408e+07
14246,ET:None,PC:1000,CO:9001,Adjustment,IO:None,CC:40001,FY24,VFP,Flash_GAAP,GL:7900400,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,-6.754470e+06
14247,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:12082,FY24,VFP,Flash_Base,GL:5540425,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,3.402402e+03


In [156]:
# Call the function that sets the PERIOD column to a Category type (to enable chronological sorting)
essbase = set_categoricals(essbase)

In [157]:
# A Jan-Dec sort order will be used, as defined in the "months"/"weeks" category that was created above and assigned to PERIOD 
if cube_name in ['SWARPT','FINSTMT']:
    if radio_compare.value == 'Yes':
        if select_comps.value == 'Before/After':
            sort_order2 = ['YEAR','PERIOD','SCEN','VER','CC','PC','IO','ACCT']
            sort_direction2 = [1,1,1,1,1,1,1,1]
            essbase = essbase.sort_values(sort_order2,ascending=sort_direction2) # Sort order is defined in the variables cell above
        elif select_comps.value == 'Scenario/Version Combos':
            sort_order2 = ['YEAR','PERIOD','CC','PC','IO','ACCT']
            sort_direction2 = [1,1,1,1,1,1]
            essbase = essbase.sort_values(sort_order2,ascending=sort_direction2) # Sort order is defined in the variables cell above
        else:
            if select_comps.value == 'Versions':
                sort_order2 = ['YEAR','PERIOD','SCEN','CC','PC','IO','ACCT']
            elif select_comps.value == 'Scenarios':
                sort_order2 = ['YEAR','PERIOD','VER','CC','PC','IO','ACCT']
            elif select_comps.value == 'Years':
                sort_order2 = ['PERIOD','VER','SCEN','CC','PC','IO','ACCT']
            sort_direction2 = [1,1,1,1,1,1,1]
            essbase = essbase.sort_values(sort_order2,ascending=sort_direction2) # Sort order is defined in the variables cell above
    else:
        sort_order2 = ['YEAR','PERIOD','SCEN','VER','CC','PC','IO','ACCT']
        sort_direction2 = [1,1,1,1,1,1,1,1]
        essbase = essbase.sort_values(sort_order2,ascending=sort_direction2) # Sort order is defined in the variables cell above
elif cube_name in ['CAPTRPT','CAPITAL']:
    if radio_compare.value == 'Yes':
        if select_comps.value == 'Before/After':
            sort_order2 = ['YEAR','PERIOD','SCEN','VER','CC','PC','PID','ACCT']
            sort_direction2 = [1,1,1,1,1,1,1,1]
            essbase = essbase.sort_values(sort_order2,ascending=sort_direction2) # Sort order is defined in the variables cell above
        elif select_comps.value == 'Scenario/Version Combos':
            sort_order2 = ['YEAR','PERIOD','CC','PC','PID','ACCT']
            sort_direction2 = [1,1,1,1,1,1]
            essbase = essbase.sort_values(sort_order2,ascending=sort_direction2) # Sort order is defined in the variables cell above
        else:
            if select_comps.value == 'Versions':
                sort_order2 = ['YEAR','PERIOD','SCEN','CC','PC','PID','ACCT']
            elif select_comps.value == 'Scenarios':
                sort_order2 = ['YEAR','PERIOD','VER','CC','PC','PID','ACCT']
            elif select_comps.value == 'Years':
                sort_order2 = ['PERIOD','VER','SCEN','CC','PC','PID','ACCT']
            sort_direction2 = [1,1,1,1,1,1,1]
            essbase = essbase.sort_values(sort_order2,ascending=sort_direction2) # Sort order is defined in the variables cell above
    else:
        sort_order2 = ['YEAR','PERIOD','SCEN','VER','CC','PC','PID','ACCT']
        sort_direction2 = [1,1,1,1,1,1,1,1]
        essbase = essbase.sort_values(sort_order2,ascending=sort_direction2) # Sort order is defined in the variables cell above
elif cube_name in ['ALLSTAFF']:
    pass
else:
    pass

In [158]:
essbase.head()

,ET,PC,CO,TYPE,IO,CC,YEAR,VER,SCEN,ACCT,FILENAME,PERIOD,DATA
22321,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:10000,FY24,VFP,Flash_Base,GL:4211000,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,121031.0
22322,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:10000,FY24,VFP,Flash_Base,GL:4221000,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,77078.0
22327,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:10000,FY24,VFP,Flash_Base,GL:4310000,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,6668.0
22328,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:10000,FY24,VFP,Flash_Base,GL:4311000,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,5539.0
22325,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:10000,FY24,VFP,Flash_Base,GL:4351000,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,27217.0


In [159]:
if all(essbase['SCEN']) == 'Authorized':
    print('Yes')
else:
    print('No')

No


In [160]:
if cube_name == 'ALLSTAFF':
    if radio_compare.value == 'Yes':
        cond1 = essbase['PERIOD'] == 'Week 53'
        essbase = essbase[cond1]
else:
    # For Current Capacity vs Working comps, filter on the months that were most recently loaded into Current Capacity
    if select_comps.value == 'Versions' and capacity_accounts.value == True:

        # Can't concatenate with PERIOD unless it's converted back to a object-type field (it's currently a Category type)
        # After that, create a new YEAR_PERIOD column
        essbase['PERIOD'] = essbase['PERIOD'].astype(object)
        essbase['YEARS_PERIODS'] = essbase['YEAR'] + '_' + essbase['PERIOD']

        # Reset the index to be the new column's values
        essbase = essbase.set_index('YEARS_PERIODS')
        
        # Open the "unique periods" file created during the Before/After comps process after the last Current Capacity load
        # Create a YEAR_PERIOD column and then set it as the index
        # Now we can use this index to filter the essbase dataframe's index and keep only the rows where the indexes match
        periods_loaded = pd.read_csv(r'C:\zDailyWork\\FINSTMT_UniquePeriods.txt', delimiter='|', header=None)
        periods_loaded['years_periods'] = periods_loaded[0] + '_' + periods_loaded[1]
        periods_loaded = periods_loaded.set_index('years_periods')
        
        # Filter on the unique year_month combinations
        index1 = essbase.index
        index2 = periods_loaded.index
        essbase = essbase[index1.isin(index2)]
        
        # Reset the essbase index back to its original values
        essbase = essbase.reset_index()
        
        # Drop the new column from the essbase dataframe
        essbase = essbase.drop('YEARS_PERIODS',axis='columns')
    
    # Filter on the forecast months
    elif filter_fcst_months.value == True:
        cond1 = essbase['PERIOD'] >= select_fcstmonth1.value
        cond2 = essbase['YEAR'] == 'FY' + str(now.year)[-2:]
        cond3 = essbase['YEAR'] != 'FY' + str(now.year)[-2:]
        essbase = essbase[(cond2 & cond1) | cond3]
    
    #Drop the BegBalance rows when Authorized is not one of the Scenario selections
    #When Authorized is the only selection, reorder the columns to match the Authorized load file; also drop the FileName column
    #cond2 = essbase['PERIOD'] != 'BegBalance'
    if all(essbase['SCEN']) == 'Authorized':
        print('Authorized is the only scenario selected.')
        essbase = essbase[['PC','YEAR','CO','VER','SCEN','PID','CC','ACCT','ET','TYPE','PERIOD','DATA','FILENAME']]
        essbase = essbase.drop(columns=['FILENAME'])
        #df = df[['A', 'B', 'C', 'D']]
    elif ~any(essbase['SCEN']) == 'Authorized':
        print('Authorized is NOT the only scenario selected.')
        cond2 = essbase['PERIOD'] != 'BegBalance'
        essbase = essbase[cond2]
    else:
        print('What is going on here?')
        print(essbase['SCEN'].head())

What is going on here?
22321    Flash_Base
22322    Flash_Base
22327    Flash_Base
22328    Flash_Base
22325    Flash_Base
Name: SCEN, dtype: object


In [161]:
essbase.info() # Check memory usage after filtering

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119166 entries, 22321 to 185140
Data columns (total 13 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   ET        119166 non-null  object  
 1   PC        119166 non-null  object  
 2   CO        119166 non-null  object  
 3   TYPE      119166 non-null  object  
 4   IO        119166 non-null  object  
 5   CC        119166 non-null  object  
 6   YEAR      119166 non-null  object  
 7   VER       119166 non-null  object  
 8   SCEN      119166 non-null  object  
 9   ACCT      119166 non-null  object  
 10  FILENAME  119166 non-null  object  
 11  PERIOD    119166 non-null  category
 12  DATA      119166 non-null  float64 
dtypes: category(1), float64(1), object(11)
memory usage: 11.9+ MB


In [162]:
essbase.head()

,ET,PC,CO,TYPE,IO,CC,YEAR,VER,SCEN,ACCT,FILENAME,PERIOD,DATA
22321,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:10000,FY24,VFP,Flash_Base,GL:4211000,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,121031.0
22322,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:10000,FY24,VFP,Flash_Base,GL:4221000,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,77078.0
22327,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:10000,FY24,VFP,Flash_Base,GL:4310000,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,6668.0
22328,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:10000,FY24,VFP,Flash_Base,GL:4311000,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,5539.0
22325,ET:None,PC:1000,CO:9001,Amount,IO:None,CC:10000,FY24,VFP,Flash_Base,GL:4351000,Users\e79230\OneDrive - Southwest Airlines\Doc...,Jan,27217.0


In [163]:
# Write the melted results to a file

if radio_compare.value == 'Yes':

    if suppress_file_name.value == True:
        essbase = essbase.drop(columns=['FILENAME'])

    essbase.to_csv (r'C:\zDailyWork\\' + cube_name + '_Compare_' + current_date + '.txt', index = False, header=True, sep='|')

else:
    if create_load_file.value == True:

        essbase = essbase.drop(columns=['FILENAME'])
    
        # For all cubes other than AllStaff, reorder the columns to match the order in the _MXL_Batch_Load load rule
        if cube_name == 'ALLSTAFF':
            pass
        elif cube_name == 'CAPITAL':
            essbase['TYPE'] = 'Amount'  # Add a dummy Type column because the standard _MXL_Batch_Load rule expects it
            essbase = essbase[['ACCT','CC','PID','CO','PC','ET','SCEN','VER','TYPE','YEAR','PERIOD','DATA']]
        elif cube_name == 'CAPTRPT':
            essbase = essbase[['ACCT','CC','PID','CO','PC','ET','SCEN','VER','TYPE','YEAR','PERIOD','DATA']]
        else:
            essbase = essbase[['ACCT','CC','IO','CO','PC','ET','SCEN','VER','TYPE','YEAR','PERIOD','DATA']]

        essbase.to_csv (r'C:\zDailyWork\\' + cube_name + '_' + current_date + '.txt', index = False, header = False, sep='|')

    
        # For the ExTO extracts, also write the results to an Excel file
        if cube_name in ['SWARPT','FINSTMT']:
            if all(essbase['ACCT'].str.startswith('HC:')) & all(essbase['TYPE'].isin(['Adjustment'])):
                # Limit the output to the current month and year entered above
                cond1 = essbase['PERIOD'] == select_fcstmonth1.value
                cond2 = essbase['YEAR'] == multiselect_year.value
                essbase = essbase[cond1 & cond2] 

                essbase.to_excel (r'C:\zDailyWork\\' + cube_name + '_' + str(select_fcstmonth1.value) + '_' + str(multiselect_year.value) + '_ExTO_Adjustments.xlsx', sheet_name = 'Load Sheet', index = False, header = True)
    
    else:
        # The file will not be loaded back into a cube, so the columns are NOT ordered for the _MXL_Batch_Load rule
        # The FileName column in included in the file 
        
        # If the dataframe contains only actuals, reorder the columns to match the load file from ETL
        if all(essbase['SCEN'] == 'Actual'):
            if all(essbase['FILENAME'].str.contains('.*CAPT_RPT.*', regex=True)):
                essbase = essbase[['PC','YEAR','CO','VER','SCEN','PID','CC','ACCT','ET','TYPE','PERIOD','DATA','FILENAME']]
                essbase = essbase.drop(columns=['FILENAME'])
            elif all(essbase['FILENAME'].str.contains('.*SWA_RPT.*', regex=True)):
                essbase = essbase[['PC','TYPE','YEAR','CO','VER','SCEN','IO','CC','ACCT','ET','PERIOD','DATA','FILENAME']]
                essbase = essbase.drop(columns=['FILENAME'])
                #PC:1000|Amount|FY22|CO:8008|Final|Actual|IO:None|CC:19910|GL:5000000|ET:None|Feb|-81305829.32
                
        # Put the columns back in the same order as the extract file(s)
        #if cube_name == 'FINSTMT':
        #    #column_headers_finstmt = ['ET','PC','CO','TYPE','IO','CC','YEAR','VER','SCEN','ACCT','BegBalance','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','FILENAME']
        #    essbase = essbase[column_headers_finstmt]
        #elif cube_name == 'SWARPT':
        #    #column_headers_swarpt = ['IO','PC','ET','YEAR','CO','TYPE','CC','VER','SCEN','ACCT','BegBalance','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','FILENAME']
        #    essbase = essbase[column_headers_swarpt]
        #elif cube_name == 'CAPITAL':
        #    #column_headers_capital = ['ET','PC','CO','YEAR','CC','PID','VER','SCEN','ACCT','BegBalance','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','FILENAME']
        #    essbase = essbase[column_headers_capital]
        #elif cube_name == 'CAPTRPT':
        #    #column_headers_captrpt = ['PID','PC','ET','TYPE','YEAR','SCEN','CO','VER','CC','ACCT','BegBalance','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','CashFlow_Period','FILENAME']
        #    essbase = essbase[column_headers_captrpt]
        
        essbase.to_csv (r'C:\zDailyWork\\' + cube_name + '_' + current_date + '.txt', index = False, header = False, sep='|')


In [164]:
# Call the function that sets the PERIOD column back to the Category type (need to sort chronologically in the next cell)
essbase = set_categoricals(essbase)

In [165]:
if radio_compare.value == 'Yes':
    if cube_name == 'ALLSTAFF':
        unique_combos = essbase[['CC','ACCT','MEASURE']].drop_duplicates()
        unique_combos = unique_combos.sort_values(by=['CC','ACCT','MEASURE'])
    elif cube_name in ['CAPTRPT','CAPITAL']:
        if select_comps.value == 'Years':
            unique_combos = essbase[['ACCT','CC','PID','PERIOD']].drop_duplicates()
            unique_combos = unique_combos.sort_values(by=['PERIOD','CC','ACCT','PID'])
            unique_combos = unique_combos[unique_combos['PERIOD'] == 'Dec'] 
        else:
            unique_combos = essbase[['ACCT','CC','PID','YEAR','PERIOD']].drop_duplicates()
            unique_combos = unique_combos.sort_values(by=['YEAR','PERIOD','CC','ACCT','PID'])
    else:
        unique_combos = essbase[['ACCT','CC','IO','YEAR','PERIOD']].drop_duplicates()
        unique_combos = unique_combos.sort_values(by=['YEAR','PERIOD','CC','ACCT','IO'])

    if cube_name in ['CAPTRPT','CAPITAL']:
        unique_pids = unique_combos[['PID']].drop_duplicates().sort_values(by='PID')

    unique_accts = unique_combos[['ACCT']].drop_duplicates().sort_values(by='ACCT')
    unique_cc = unique_combos[['CC']].drop_duplicates().sort_values(by='CC')
    
    if cube_name != 'ALLSTAFF':
        if not select_comps.value == 'Years':
            unique_periods = unique_combos[['YEAR','PERIOD']].drop_duplicates().sort_values(by=['YEAR','PERIOD'])
    

In [166]:
if radio_compare.value == 'Yes':
    unique_combos.to_csv (r'C:\zDailyWork\\' + cube_name + '_UniqueCombos_' + current_date + '.txt', index = False, header=False, sep='|')
    unique_accts.to_csv (r'C:\zDailyWork\\' + cube_name + '_UniqueAccounts_' + current_date + '.txt', index = False, header=False, sep='|')
    unique_cc.to_csv (r'C:\zDailyWork\\' + cube_name + '_UniqueCostCenters_' + current_date + '.txt', index = False, header=False, sep='|')
    
    if cube_name in ['CAPTRPT','CAPITAL']:
        unique_pids.to_csv (r'C:\zDailyWork\\' + cube_name + '_UniquePIDs_' + current_date + '.txt', index = False, header=False, sep='|')

    if cube_name != 'ALLSTAFF':
        if not select_comps.value == 'Years':
            unique_periods.to_csv (r'C:\zDailyWork\\' + cube_name + '_UniquePeriods_' + current_date + '.txt', index = False, header=False, sep='|')
            
            # For capacity loads only, create a generic "periods loaded" file (without the date) for the Before/After file set
            # It will be used during the Current Capacity vs Working comps
            if 'Current Capacity' in list(multiselect_version.value) and select_comps.value == 'Before/After':
                unique_periods.to_csv (r'C:\zDailyWork\\' + cube_name + '_UniquePeriods' + '.txt', index = False, header=False, sep='|')
            